# Prepare

In [1]:
#!pip install -e ..
from altair.vegalite.v4.api import FacetChart, Chart, LayerChart
from sklearn.preprocessing import LabelEncoder
import bulwark.checks as ck
from bayes_window.fake_spikes import generate_fake_spikes
from bayes_window.visualization import fake_spikes_explore, plot_data_and_posterior
from bayes_window import models
from bayes_window.fitting import fit_numpyro
from bayes_window.utils import add_data_to_posterior

trans = LabelEncoder().fit_transform



/home/m/anaconda3/envs/tf/lib/python3.6/site-packages/elephant/pandas_bridge.py:22: DeprecationWarning: pandas_bridge module will be removed in Elephant v0.8.x
  DeprecationWarning)


# Make some data


In [2]:
df, df_monster, index_cols, firing_rates = generate_fake_spikes(n_trials=2,
                                                                n_neurons=8,
                                                                n_mice=4,
                                                                dur=7, )

# Estimate model

In [3]:
#y = list(set(df.columns) - set(index_cols))[0]
y='isi'
trace = fit_numpyro(y=df[y].values,
                    stim_on=(df['stim']).astype(int).values,
                    treat=trans(df['neuron']),
                    subject=trans(df['mouse']),
                    progress_bar=True,
                    model=models.model_hier_normal_stim,
                    n_draws=100, num_chains=1, )

sample: 100%|██████████| 1100/1100 [00:07<00:00, 141.87it/s, 23 steps of size 1.62e-01. acc. prob=0.90]


n(Divergences) = 0


# Add data back

In [4]:
df_both = add_data_to_posterior(df,
                                trace=trace,
                                y=y,
                                index_cols=['neuron', 'stim', 'mouse', ],
                                condition_name='stim',
                                b_name='b_stim_per_condition',  # for posterior
                                group_name='neuron'  # for posterior
                                )

# Plot data and posterior

In [5]:
plot_data_and_posterior(df_both, y=f'{y} diff', x='neuron',color='mouse',title=y,hold_for_facet=False)

alt.LayerChart(...)